In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [17]:
# 创建一个空的DataFrame来存储数据
df = pd.DataFrame(columns=['topic', 'feature', 'describe'])

In [18]:
urls_and_topics = {
    "http://www.yusoo.com.tw/mobile.php?dir=Palmistry&web=chiromancy_information&kw=heart_line": "感情線(天紋)",
    "http://www.yusoo.com.tw/mobile.php?dir=Palmistry&web=chiromancy_information&kw=life_line": "生命線(地紋)",
    "http://www.yusoo.com.tw/mobile.php?dir=Palmistry&web=chiromancy_information&kw=head_line": "智慧線(人紋)",
    "http://www.yusoo.com.tw/mobile.php?dir=Palmistry&web=chiromancy_information&kw=mariage_line": "婚姻線(家風紋)",
    "http://www.yusoo.com.tw/mobile.php?dir=Palmistry&web=chiromancy_information&kw=children": "子女線(家風紋)",
    "http://www.yusoo.com.tw/mobile.php?dir=Palmistry&web=chiromancy_information&kw=fate_line": "事業線(天喜紋)",
    "http://www.yusoo.com.tw/mobile.php?dir=Palmistry&web=chiromancy_information&kw=girdle_of_venvs": "金星帶(橫財紋)",
    "http://www.yusoo.com.tw/mobile.php?dir=Palmistry&web=chiromancy_information&kw=apollo_line": "太陽線(高扶紋)",
    "http://www.yusoo.com.tw/mobile.php?dir=Palmistry&web=chiromancy_information&kw=mentor": "火星線(貴人紋)",
    "http://www.yusoo.com.tw/mobile.php?dir=Palmistry&web=chiromancy_information&kw=bracelet": "手腕線(動脈紋)",
    "http://www.yusoo.com.tw/mobile.php?dir=Palmistry&web=chiromancy_information&kw=health_line": "健康線",
    "http://www.yusoo.com.tw/mobile.php?dir=Palmistry&web=chiromancy_information&kw=travel": "旅行線"
}

In [19]:
# 遍历URL和主题
for url, topic in urls_and_topics.items():
    for page in range(1, 7):  # 循环遍历所有6页
        if page == 1:
            full_url = url
        else:
            full_url = f"{url}&page={page}"

        # 发起HTTP GET请求
        response = requests.get(full_url)
        time.sleep(0.5)
        # 检查请求是否成功
        if response.status_code == 200:
            # 使用BeautifulSoup解析HTML
            soup = BeautifulSoup(response.content, 'html.parser')

            # 查找所有class为'pal_list'的div元素
            divs = soup.find_all('div', class_='pal_list')

            # 存储当前页的数据
            page_data = []

            # 遍历这些div元素
            for div in divs:
                # 获取标题
                title = div.find('h3').get_text()
                # 获取段落内容
                paragraph = div.find('p').get_text().replace('\n', '')

                # 将数据添加到列表中
                page_data.append({
                    'topic': topic,
                    'feature': title,
                    'describe': paragraph
                })

            # 将当前页的数据转化为DataFrame并与主DataFrame合并
            df = pd.concat([df, pd.DataFrame(page_data)], ignore_index=True)
        else:
            print(f'请求{topic}的第{page}页失败，状态代码:', response.status_code)

In [20]:
df.to_csv('../data/data1.csv', index=False, encoding='utf-8')